# Define folder methods

In [1]:
# prompt: create_folder_in_drive - define a function that create the folder /stock-rl in my google drive if it doesn't already exist. paramatrize the name of the folder

import os
from google.colab import drive
drive.mount('/content/drive')

def create_folder_in_drive(folder_name):
  """Creates a folder in Google Drive if it doesn't exist.

  Args:
    folder_name: The name of the folder to create.
  """
  drive_path = '/content/drive/My Drive/'  # Root directory in Google Drive
  folder_path = os.path.join(drive_path, folder_name)

  if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_name}' created in Google Drive.")
  else:
    print(f"Folder '{folder_name}' already exists in Google Drive.")

# Example usage:
create_folder_in_drive('stock-rl')


Mounted at /content/drive
Folder 'stock-rl' already exists in Google Drive.


In [2]:
# prompt: get_from_drive - define a function that get the file from google drive to local drive, under the folder /stock-rl. if the file already exists in the local drive, remove the old file. parametrize the file name and path name. do not change the name of the file

def get_from_drive(drive_filename, local_filename='', drive_folder='/stock-rl'):
  """Gets a file from Google Drive to the local drive, overwriting if it exists.

  Args:
    drive_filename: Name of the file in Google Drive.
    local_filename: Path to save the file locally.
    drive_folder: Folder in Google Drive to get from (default: '/stock-rl').
  """
  drive_path = '/content/drive/My Drive/' + drive_folder + '/' + drive_filename
  if not local_filename:
    local_filename = drive_filename

  if not os.path.exists(drive_path):
    print(f"File '{drive_filename}' does not exist in Google Drive.")
    return

  if os.path.exists(local_filename):
    os.remove(local_filename)
    print(f"Existing local file '{local_filename}' removed.")

  !cp "{drive_path}" "{local_filename}"
  print(f"File '{drive_filename}' retrieved from Google Drive to '{local_filename}'.")

# Example usage:
get_from_drive('stock_data.h5', 'stock_data.h5')
get_from_drive('stock_data_index.pkl', 'stock_data_index.pkl')


File 'stock_data.h5' retrieved from Google Drive to 'stock_data.h5'.
File 'stock_data_index.pkl' retrieved from Google Drive to 'stock_data_index.pkl'.


In [6]:
# prompt: load_dataframe_from_h5 - define a function that read the data from filename into a variable data2, and then restore the column names and row index from stock_data_index.pkl

import pandas as pd
import h5py
import pickle
def load_dataframe_from_h5(filename, index_filename):
  """Loads a pandas DataFrame from an HDF5 file and restores its index.

  Args:
    filename: The name of the HDF5 file containing the DataFrame data.
    index_filename: The name of the file containing the DataFrame index information.

  Returns:
    The loaded pandas DataFrame with restored index.
  """
  try:
    # Load the DataFrame data from the HDF5 file
    with h5py.File(filename, 'r') as hf:
      data2 = pd.DataFrame(hf['data'][:])

    # Load the index information from the pickle file
    with open(index_filename, 'rb') as f:
      index_info = pickle.load(f)

    # Restore the column names and row index
    data2.columns = index_info['columns']
    data2.index = index_info['index']

    print(f"DataFrame loaded from '{filename}' with restored index.")
    return data2

  except Exception as e:
    print(f"Error loading DataFrame: {e}")
    return None

# Example usage:
data = load_dataframe_from_h5('stock_data.h5', 'stock_data_index.pkl')
print(data.head())

DataFrame loaded from 'stock_data.h5' with restored index.
Price      Adj Close                                                      \
Ticker        360.AX A2M.AX AAI.AX AFI.AX AGL.AX AIA.AX    ALD.AX ALL.AX   
Date                                                                       
1988-01-29       NaN    NaN    NaN    NaN    NaN    NaN  0.574315    NaN   
1988-02-01       NaN    NaN    NaN    NaN    NaN    NaN  0.574315    NaN   
1988-02-02       NaN    NaN    NaN    NaN    NaN    NaN  0.574315    NaN   
1988-02-03       NaN    NaN    NaN    NaN    NaN    NaN  0.574315    NaN   
1988-02-04       NaN    NaN    NaN    NaN    NaN    NaN  0.574315    NaN   

Price                           ... Volume                                     \
Ticker        ALQ.AX    AMC.AX  ... WEB.AX WES.AX WGX.AX WHC.AX WOR.AX WOW.AX   
Date                            ...                                             
1988-01-29  0.056859  0.313313  ...    NaN    0.0    NaN    NaN    NaN    NaN   
1988-02-

In [7]:
# prompt: save_dataframe_to_h5 - using h5py, define a function that store the dataframe on my hard drive. in the case where a filename already exists, remove the old file

import h5py

def save_dataframe_to_h5(dataframe, filename):
  """Saves a pandas DataFrame to an HDF5 file.

  Args:
    dataframe: The pandas DataFrame to save.
    filename: The name of the HDF5 file to create.
  """
  try:
    # Check if the file already exists and remove it
    if os.path.exists(filename):
      os.remove(filename)
      print(f"Existing file '{filename}' removed.")

    # Save the DataFrame to the HDF5 file
    with h5py.File(filename, 'w') as hf:
      hf.create_dataset('data', data=dataframe.to_numpy())
    print(f"DataFrame saved to '{filename}'.")

  except Exception as e:
    print(f"Error saving DataFrame: {e}")

# Example usage:
save_dataframe_to_h5(data, 'stock_data.h5')


Existing file 'stock_data.h5' removed.
DataFrame saved to 'stock_data.h5'.


In [8]:
# prompt: store_dataframe_index - using pickle, define a function that store the columns and row index of the dataframe

import pickle

def store_dataframe_index(dataframe, filename):
  """Stores the columns and row index of a pandas DataFrame using pickle.

  Args:
    dataframe: The pandas DataFrame.
    filename: The name of the file to store the index information.

  Remarks:
    The stored object contains these keys: 'columns', 'index'.
  """
  index_info = {
      'columns': dataframe.columns,
      'index': dataframe.index
  }
  with open(filename, 'wb') as f:
    pickle.dump(index_info, f)
  print(f"DataFrame index information saved to '{filename}'.")

# Example usage:
store_dataframe_index(data, 'stock_data_index.pkl')


DataFrame index information saved to 'stock_data_index.pkl'.


In [9]:
# prompt: save_to_drive - define a function that store stock_data.h5 and columns.pkl into my google drive, under the folder /stock-rl. if the file already exists in the google drive, remove the old file. parametrize the path name

def save_to_drive(local_filename, drive_filename='', drive_folder='/stock-rl'):
  """Saves a local file to Google Drive, overwriting if it exists.

  Args:
    local_filename: Path to the local file.
    drive_filename: Name of the file in Google Drive.
    drive_folder: Folder in Google Drive to save to (default: '/stock-rl').
  """
  drive_path = '/content/drive/My Drive/' + drive_folder + '/' + drive_filename
  if not drive_filename:
    drive_filename = local_filename

  if not os.path.exists(local_filename):
    print(f"Local file '{local_filename}' does not exist.")
    return

  if os.path.exists(drive_path):
    os.remove(drive_path)
    print(f"Existing file '{drive_filename}' removed from Google Drive.")

  !cp "{local_filename}" "{drive_path}"
  print(f"File '{local_filename}' saved to Google Drive as '{drive_filename}'.")

# Example usage:
save_to_drive('stock_data.h5', 'stock_data.h5')
save_to_drive('stock_data_index.pkl', 'stock_data_index.pkl')


Existing file 'stock_data.h5' removed from Google Drive.
File 'stock_data.h5' saved to Google Drive as 'stock_data.h5'.
Existing file 'stock_data_index.pkl' removed from Google Drive.
File 'stock_data_index.pkl' saved to Google Drive as 'stock_data_index.pkl'.


# Data cleaning

In [10]:
data

Price       Adj Close                                                    \
Ticker         360.AX A2M.AX     AAI.AX AFI.AX AGL.AX AIA.AX     ALD.AX   
Date                                                                      
1988-01-29        NaN    NaN        NaN    NaN    NaN    NaN   0.574315   
1988-02-01        NaN    NaN        NaN    NaN    NaN    NaN   0.574315   
1988-02-02        NaN    NaN        NaN    NaN    NaN    NaN   0.574315   
1988-02-03        NaN    NaN        NaN    NaN    NaN    NaN   0.574315   
1988-02-04        NaN    NaN        NaN    NaN    NaN    NaN   0.574315   
...               ...    ...        ...    ...    ...    ...        ...   
2024-08-05  15.170000   6.79  44.247307   7.30  10.29   6.73  32.480000   
2024-08-06  15.160000   6.67  43.699196   7.38  10.46   6.70  32.209999   
2024-08-07  15.100000   6.64  44.954865   7.37  10.66   6.75  32.369999   
2024-08-08  14.870000   6.58  44.097820   7.43  10.75   6.78  32.080002   
2024-08-09  17.559999   6.77  46.330002   7.46  10.82   6.80  32.240002   

Price                                        ...     Volume             \
Ticker         ALL.AX     ALQ.AX     AMC.AX  ...     WEB.AX     WES.AX   
Date                                         ...                         
1988-01-29        NaN   0.056859   0.313313  ...        NaN        0.0   
1988-02-01        NaN   0.056859   0.313313  ...        NaN        0.0   
1988-02-02        NaN   0.056859   0.313313  ...        NaN        0.0   
1988-02-03        NaN   0.056859   0.313313  ...        NaN        0.0   
1988-02-04        NaN   0.056859   0.313313  ...        NaN        0.0   
...               ...        ...        ...  ...        ...        ...   
2024-08-05  49.750000  14.830000  15.930000  ...  2048039.0  1717129.0   
2024-08-06  51.009998  14.860000  16.010000  ...  1322366.0  1578117.0   
2024-08-07  51.099998  14.930000  16.059999  ...  1036201.0  1119109.0   
2024-08-08  51.759998  14.680000  16.030001  ...  1771014.0  1131089.0   
2024-08-09  51.889999  15.110000  16.200001  ...  2076627.0   869480.0   

Price                                                                        \
Ticker          WGX.AX     WHC.AX     WOR.AX     WOW.AX    WTC.AX    XRO.AX   
Date                                                                          
1988-01-29         NaN        NaN        NaN        NaN       NaN       NaN   
1988-02-01         NaN        NaN        NaN        NaN       NaN       NaN   
1988-02-02         NaN        NaN        NaN        NaN       NaN       NaN   
1988-02-03         NaN        NaN        NaN        NaN       NaN       NaN   
1988-02-04         NaN        NaN        NaN        NaN       NaN       NaN   
...                ...        ...        ...        ...       ...       ...   
2024-08-05  15096871.0  7195263.0  3066641.0  2621990.0  771835.0  561988.0   
2024-08-06  18273410.0  5196958.0  3658677.0  2479244.0  916998.0  362183.0   
2024-08-07   8583281.0  4028573.0  1878453.0  1594304.0  462445.0  302424.0   
2024-08-08   5530053.0  4254866.0  1506417.0  1847138.0  337869.0  217364.0   
2024-08-09   7192550.0  3990704.0  1036245.0  1632297.0  284666.0  271887.0   

Price                              
Ticker         YAL.AX      ZIP.AX  
Date                               
1988-01-29        NaN         NaN  
1988-02-01        NaN         NaN  
1988-02-02        NaN         NaN  
1988-02-03        NaN         NaN  
1988-02-04        NaN         NaN  
...               ...         ...  
2024-08-05  3077096.0  35182335.0  
2024-08-06  3925725.0  24683158.0  
2024-08-07  2059237.0  16668273.0  
2024-08-08  2904199.0  11855552.0  
2024-08-09  1349502.0  17090199.0  

[9373 rows x 840 columns]

In [11]:
# prompt: Define an RSI calculator. Return a new dataframe that is the RSI column but do not append it to the original dataframe

import pandas as pd

def calculate_rsi(data, window=14):
  """
  Calculates the Relative Strength Index (RSI) for a given DataFrame.

  Args:
    data: A pandas DataFrame with a 'Close' column representing stock closing prices.
    window: The period for calculating RSI (default is 14).

  Returns:
    A new DataFrame containing only the RSI column.
  """

  delta = data['Close'].diff()
  gain = delta.where(delta > 0, 0)
  loss = -delta.where(delta < 0, 0)

  avg_gain = gain.rolling(window=window).mean()
  avg_loss = loss.rolling(window=window).mean()

  rs = avg_gain / avg_loss
  rsi = 100 - (100 / (1 + rs))

  return pd.DataFrame({'RSI': rsi})

# Example usage:
# Assuming you have a DataFrame called 'data' with a 'Close' column
# rsi_df = calculate_rsi(data)
# print(rsi_df)



In [12]:
# prompt: Define a KDJ calculator. Return a new dataframe that is the calculated column but do not append it to the original dataframe. Pass in the parameters into the method with default values.

import pandas as pd

# Since this is a linear combination of values in data, this is probably not a needed feature.
def calculate_kdj(data, n=9, m1=3, m2=3):
  """
  Calculates the KDJ indicator for a given DataFrame.

  Args:
    data: A pandas DataFrame with 'High', 'Low', and 'Close' columns representing
          stock high, low, and closing prices.
    n: The period for calculating RSV (default is 9).
    m1: The period for calculating K (default is 3).
    m2: The period for calculating D (default is 3).

  Returns:
    A new DataFrame containing the calculated K, D, and J values.
  """

  low_n = data['Low'].rolling(window=n).min()
  high_n = data['High'].rolling(window=n).max()
  rsv = ((data['Close'] - low_n) / (high_n - low_n)) * 100
  rsv.fillna(50, inplace=True)

  k = rsv.ewm(alpha=1 / m1, adjust=False).mean()
  d = k.ewm(alpha=1 / m2, adjust=False).mean()
  j = 3 * k - 2 * d

  return pd.DataFrame({'K': k, 'D': d, 'J': j})

# Example usage:
# Assuming you have a DataFrame called 'data' with 'High', 'Low', and 'Close' columns
# kdj_df = calculate_kdj(data)
# print(kdj_df)


In [21]:
# prompt: For each stock, calculate the RSI value and put it into a new dataframe

import pandas as pd
# Assuming 'data' is your DataFrame containing stock data with a 'Close' column for each stock.
# You might need to adjust the code depending on how your data is organized.

rsi_data = pd.DataFrame()  # Create an empty DataFrame to store RSI values

for stock_symbol in data.columns:
  if 'Close' in stock_symbol: # Check if the column is a 'Close' column for a stock
    stock_close_data = data[[stock_symbol]]
    stock_close_data.columns = ['Close'] # Rename the column to 'Close'
    stock_rsi = calculate_rsi(stock_close_data)
    # stock_symbol is like ('Close', '360.AX')
    # Rename RSI column to ('RSI', '360.AX')
    stock_rsi.columns = [('RSI', stock_symbol[1])]
    rsi_data = pd.concat([rsi_data, stock_rsi], axis=1)


print(rsi_data.tail())



            (RSI, 360.AX)  (RSI, A2M.AX)  (RSI, AAI.AX)  (RSI, AFI.AX)  \
Date                                                                     
2024-08-05      37.293748      47.972975      25.404315      49.411779   
2024-08-06      33.968813      37.762240      24.496424      48.192785   
2024-08-07      37.404592      40.000000      30.210216      51.948036   
2024-08-08      37.333336      40.000000      28.726447      58.974359   
2024-08-09      59.615381      53.284677      36.294036      62.820505   

            (RSI, AGL.AX)  (RSI, AIA.AX)  (RSI, ALD.AX)  (RSI, ALL.AX)  \
Date                                                                     
2024-08-05      47.272702      27.631576      35.619027      37.323945   
2024-08-06      52.459037      27.272716      31.001873      42.580639   
2024-08-07      60.000000      33.333333      36.363615      42.019537   
2024-08-08      64.137937      35.802484      36.659892      42.765260   
2024-08-09      70.922022      38.750

In [22]:
# prompt: Define a method to do: For each stock, calculate the RSI value and append it to the original dataframe

import pandas as pd
def calculate_rsi_and_append(data, window=14):
  """
  Calculates the RSI for each stock in the DataFrame and appends it as a new column.

  Args:
    data: A pandas DataFrame with 'Close' columns for each stock.
    window: The period for calculating RSI (default is 14).

  Returns:
    The original DataFrame with added RSI columns for each stock.
  """

  for stock_symbol in data.columns:
    if 'Close' in stock_symbol:
      stock_close_data = data[[stock_symbol]]
      stock_close_data.columns = ['Close']
      stock_rsi = calculate_rsi(stock_close_data)
      stock_rsi.columns = [('RSI', stock_symbol[1])]
      data = pd.concat([data, stock_rsi], axis=1)

  return data

# Example usage:
# data_with_rsi = calculate_rsi_and_append(data)
# print(data_with_rsi.tail())


In [23]:
# prompt: Append RSI to the original dataframe, and save it into the file stock_data_processed.h5 and save it into google drive

data_with_rsi = calculate_rsi_and_append(data)
save_dataframe_to_h5(data_with_rsi, 'stock_data_processed.h5')
store_dataframe_index(data_with_rsi, 'stock_data_processed_index.pkl')
save_to_drive('stock_data_processed.h5', 'stock_data_processed.h5')
save_to_drive('stock_data_processed_index.pkl', 'stock_data_processed_index.pkl')


DataFrame saved to 'stock_data_processed.h5'.
DataFrame index information saved to 'stock_data_processed_index.pkl'.
File 'stock_data_processed.h5' saved to Google Drive as 'stock_data_processed.h5'.
File 'stock_data_processed_index.pkl' saved to Google Drive as 'stock_data_processed_index.pkl'.
